In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics
from sklearn.model_selection import cross_val_score

2017-08-25 08:07:47,139 : INFO : 'pattern' package found; tag filters are available for English
/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Dictionary features

In [2]:
personalities = pd.read_csv('~/personality-normalized-word2vec-norm.csv', encoding='utf-8')

In [3]:
personalities.shape

(1039, 186)

In [4]:
personalities.head()

,Unnamed: 0,file_name,formatted_text,X,sentences,items,chars,allTokens,wordTokens,skip,...,extraversion_m,agreeabeness_m,conscientiousness_m,neuroticism_m,openness_m,extraversion_ober_2,agreeableness_ober_2,conscientiousness_ober_2,neuroticism_ober_2,openness_ober_2
0,1,100,ajudando porque a zuzu é um amor e tem a voz f...,1,230,2032,9649,968,609,0.0089,...,0,0,0,1,1,0,0,0,1,1
1,2,1000,Chega de saudade ! A festa brasileira mais b...,2,240,2538,12020,965,870,0.0004,...,0,0,1,1,0,0,-1,-1,1,0
2,3,1001,Se tem um filme em todas as galáxias existen...,3,247,4245,19933,1503,1353,0,...,0,1,0,0,0,-1,1,-1,-1,-1
3,4,1002,"Sobre ter vizinhas prendadas e fofas , que t...",4,206,3982,19492,1328,1151,0.0005,...,1,0,1,1,0,1,0,1,-1,-1
4,5,1003,Nostalgia Agora eu entendi Só rindo da pia...,5,136,805,4448,477,401,0,...,1,1,0,1,1,1,1,-1,-1,1


In [5]:
personalities = personalities.convert_objects(convert_numeric=True)

/usr/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [6]:
personalities.agreeabeness_m.value_counts()

1    542
0    497
Name: agreeabeness_m, dtype: int64

In [7]:
pre_proc_attr = ['skip', 'compound', 'hashtags', 'links', 'punct', 'questions', 'exclam', 'numbers', 'upcase', 'lowcase', 'firstup', 'pt.lexicon', 'added', 'verb.pro', 'names', 'en.lexicon', 'rewrite', 'mispell', 'foreign', 'laugh', 'emph', 'echars', 'unkn']

In [8]:
grammar_attr = ['m', 'f', 's', 'p', 'aument', 'dimin', 'superlat', 'N', 'A', 'PREP', 'CONJ', 'ADV', 'PREFIX', 'SIGLA', 'ABREV', 'INTERJ', 'DET', 'def', 'indef', 'NUM', 'numC', 'numO', 'numM', 'numF', 'PRO', 'proDem', 'proIndef', 'proRel', 'proInterr', 'proTrat', 'proPoss', 'proPess', 'acusativa', 'dativa', 'nominativa', 'obliqua', 'reflexa', 'p1', 'p2', 'p3', 'V', 'VW', 'VG', 'VK', 'VP', 'VI', 'VJ', 'VF', 'VQ', 'VS', 'VT', 'VU', 'VY', 'VC', 'V1s', 'V2s', 'V3s', 'V1p', 'V2p', 'V3p']

In [9]:
liwc_attr = ['X1funct' ,'X2pronoun' ,'X3ppron','X4i','X5we','X6you','X7shehe','X8they','X9ipron','X10article','X11verb','X12auxverb','X13past','X14present','X15future','X16adverb','X17preps','X18conj','X19negate','X20quant','X21number','X22swear','X23social','X24family','X25friend','X26humans','X27affect','X28posemo','X29negemo','X30anx','X31anger','X32sad','X33cogmech','X34insight','X35cause','X36discrep','X37tentat','X38certain','X39inhib','X40incl','X41excl','X42percept','X43see','X44hear','X45feel','X46bio','X47body','X48health','X49sexual','X50ingest','X51relativ','X52motion','X53space','X54time','X55work','X56achieve','X57leisure','X58home','X59money','X60relig','X61death','X62assent','X63nonfl', 'X64filler']

In [26]:
all_attr = personalities[:,6:157]

TypeError: unhashable type

In [25]:
all_attr.head()

,Unnamed: 0,file_name,formatted_text,X,sentences,items,chars,allTokens,wordTokens,skip,...,extraversion_m,agreeabeness_m,conscientiousness_m,neuroticism_m,openness_m,extraversion_ober_2,agreeableness_ober_2,conscientiousness_ober_2,neuroticism_ober_2,openness_ober_2
0,1,100,ajudando porque a zuzu é um amor e tem a voz f...,1,230,2032,9649,968,609,0.0089,...,0,0,0,1,1,0,0,0,1,1


In [11]:
def remove_nan(atributes):
    personalities_attributes = np.array(personalities[atributes])
    personalities_attributes[np.isnan(personalities_attributes)] = 0
    return personalities_attributes

In [16]:
data = remove_nan(all_attr.keys())

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
clf = RandomForestClassifier(max_depth=2, random_state=42, criterion= "gini", class_weight="balanced")

## Extraversion

In [ ]:
scores = cross_val_score(clf, data, personalities['extraversion_m'], cv=10, scoring='f1_macro')

In [ ]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## Agreableness

In [ ]:
scores = cross_val_score(clf, data, personalities['agreeabeness_m'], cv=10, scoring='f1_macro')

In [ ]:
scores

In [ ]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## Conscientiousness

In [ ]:
scores = cross_val_score(clf, data, personalities['conscientiousness_m'], cv=10, scoring='f1_macro')

In [ ]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## Neuroticism

In [ ]:
scores = cross_val_score(clf, data, personalities['neuroticism_m'], cv=10, scoring='f1_macro')

In [ ]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## Openness

In [ ]:
scores = cross_val_score(clf, data, personalities['openness_m'], cv=10, scoring='f1_macro')

In [ ]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))